# Dataset reading

The analyzed dataset is a set of english correctly spelled words as well as variations of their incorrect spelling, collected by Wikipedia editors. File with the data wad downloaded from this [article](https://www.kaggle.com/datasets/bittlingmayer/spelling?resource=download&select=wikipedia.txt).

Also, there are interesting ways to generate misspelled words from the correct ones described in this [article](https://www.ijcaonline.org/archives/volume176/number27/yunus-2020-ijca-920288.pdf). However, in this work I have so far stopped at the Wikipedia dataset described above.

Most of the spell checking tools support the ability to find errors and correct them in English texts, so a set of words in English was selected for testing. As a sequence of words, both incorrect words and correct ones will be submitted to the checkers in order to be able to track how the analyzed tools react and possibly correct the right spelled words.

Because a different number of potential correct spellings are presented for each misspelled word in the dataset, it's not possible to use the *read_csv* method from *pandas* library, where it's required the same number of columns-features for each row. So the dataset will be processed line by line.

TODO: можно несколько датасетов использовать! все, которые на том сайте есть по крайней мере под парсинг подойдут.

TODO: Можно потом сделать одну таблицу, где собрать все метрики по каждому инструменту (в readme)

In [106]:
def read_misspells_dataset(path_to_misspells_file: str) -> dict:
    """

    Parameters
    ----------
    path_to_misspells_file : str
        Path to the dataset file (may have any format).

    Returns
    -------
    dataset_dict : dict
        Dictionary from misspelled dataset word to list of its possible correct spellings.

    """
    # reading dataset file
    with open(path_to_misspells_file, 'r') as dataset_file:
        dataset_lines = dataset_file.readlines()

    # delete '\n' symbols
    dataset_lines = [line.strip() for line in dataset_lines]

    # filling dataset dictionary
    dataset_dict = {}
    for word_line in dataset_lines:
        line_words = word_line.split()
        correct_word = line_words[0][:-1]  # removing ':'

        misspellings = []
        if len(line_words) > 1:
            misspellings = line_words[1:]
        dataset_dict[correct_word] = misspellings

    return dataset_dict

In [107]:
# path_to_misspells_dataset = "data/wikipedia_misspells.txt"
# misspells_dict = read_misspells_dataset(path_to_misspells_dataset)
# print(misspells_dict)
# TODO

Now let's build a dataset that we will give to checkers. This will be a string of incorrect words in which the correct words are also inserted. For some spell checking tools, it will be necessary to divide this line into spaces and give a list of individual words.

In [108]:
def build_correct_and_misspell_sentence(misspells_dict: dict):
    misspell_str = ""
    correct_str = ""
    invalids_in_misspell_str = 0

    for correct_word in misspells_dict:
        misspelled_forms = misspells_dict[correct_word]
        invalids_in_misspell_str += len(misspelled_forms)
        misspelled_forms_str = " ".join(misspelled_forms)
        misspell_str += correct_word + ' ' + misspelled_forms_str + ' '

        correct_forms = " ".join([correct_word] * len(misspelled_forms))
        correct_str += correct_word + ' ' + correct_forms + ' '

    return correct_str, misspell_str, invalids_in_misspell_str

In [109]:
# correct_sentence, misspell_sentence = build_correct_and_misspell_sentence()
# print(correct_sentence)
# print(misspell_sentence)
# TODO

# Metrics

## Metrics group
Metrics for spell checker evaluating can be divided into 3 groups:
1. Metrics that evaluate the tool's ability to **classify words as right spelled or misspelled**
2. Metrics that evaluate the tool's ability to **correct misspelled words**
3. Others

## Metrics of each group
1. To understand the 1st metrics group, this [article](https://gerhard.pro/files/PublicationVanHuyssteenEiselenPuttkammer2004.pdf) can be helpful. I relied on this paper, but changed the approach to putting positive and negative labels, based on the idea that we are still looking for misspelled words (as in the problem of detecting cancerous tumors, we usually label cancerous cases as positives). I chose standard metrics without the additional subdivision, described in the article.

    Thus,  there will be used the following metrics:
    * **Recall**
    * **Precision**
    * **Classifying Accuracy**

2. In the second metrics group I will use these metrics:
    * **Percent of the misspelled words**, that were **correctly fixed** by spellchecker
    * **Percent of non-fixed misspelled words**, but for which the right decision was **in top-5** spellchecker suggested word **candidates**
    * **Percent of** originally correct spelled words that were **broken** by the checker

3. In 3rd group there was put one metric:
    * **Speed**

## Formulas and explanation of each metric

As we are looking for misspelled words, positive labels will be set for invalid words with the misspellings and negative labels -- for valid correct spelled words.

In that way:
* **True positives (tp)** -- invalid words, recognized by spelling checker as misspelled.
* **False positives (fp)** -- valid words, recognized by checker as misspelled.
* **True negatives (tn)** -- valid words, recognized by checker as right spelled.
* **False negatives (fn)** -- invalid words, recognized by checker as right spelled.

### 1. Recall

Recall -- number of invalid words that were recognized by the checker as misspelled (true positives), in relation to the total number of invalid words in the text (sum of true positives and false negatives)

$$ recall = \frac{tp}{tp + fn} $$

The ideal for the spelling checker -- to recognize all invalid words as misspelled $\Rightarrow$ to get as high recall as possible, the closer to 100% the better.
Recall indicates comprehensiveness of the lexicon of the spelling checker and whether the spelling checker lexicon contains any erroneous words.

### 2. Precision

Precision -- number of words, that were marked by checker as misspelled and which are really incorrect, in relation to the total number of words, that checker marked as misspelled (sum of true positives and false positives).

$$ precision = \frac{tp}{tp + fp} $$

The ideal for the checker -- to recognize all invalid and only invalid words as misspelled $\Rightarrow$ to get as high precision as possible, the closer to 100% the better.
Precision indicates how accurate is the spelling checker in assigning misspell flag.

### 3. Identifying Accuracy

Accuracy shows what the spelling checker does right (the sum of true positives and true negatives) in terms of everything the spelling checker does (all true and false positives and negatives).

$$ identifyingAccuracy = \frac{tp + tn}{tp + fp + tn + fn} $$

This gives a good overall view of the competence of a checker, since it determines how accurate a spelling checker is at performing the errand it was sent on. The better spell checking tool works, the higher accuracy $\Rightarrow$ the ideal for the checker -- to get 100% accuracy.

### 4. Percent of words that are invalid after checker work
$$ invalidsAfterCheckerPerc = \frac{invalidsAfterCheckerWork}{nWordsInText} $$
The ideal for spellchecker -- 0% -- when all words in text after checker work are valid. The lower, the better.

### 5. Percent of correctly fixed misspellings

$$ fixedMisspellingsPerc = \frac{fixedMisspellings}{allMisspellsInText} $$

The ideal for spellchecker -- 100%. The higher, the better.

### 6. Percent of non-fixed misspellings but with right correction in top-5 candidates

$$ notFixedButInTop5Perc = \frac{notFixedButCorrectionInTop5Candidates}{notFixedMisspells} $$

The ideal for spellchecker -- 100% -- when all not fixed misspells had right correction in top-5 candidates. The higher, the better.

### 7. Percent of broken valid words

$$ brokenValidsPerc = \frac{brokenValids}{allOriginallyValids} $$

The ideal for spellchecker -- 0% -- when he didn't break a single word. The lower, the better.

### 8. Speed

Calculates number of words per second and shows how fast the spell checking tool copes with data processing. Can be calculated as the total time spent by the spell checker for processing the dataset, divided by the number of words processed by it:

$$ speed = \frac{numberOfWordsInText}{totalCheckerTextProcessingTime} $$

The higher the speed of the checker, the less the user will have to wait $\Rightarrow$ the higher speed the better.

In [110]:
def calculate_main_metrics(tp: int, fp: int, tn: int, fn: int):
    recall = float(tp) / (tp + fn)
    precision = float(tp) / (tp + fp)
    accuracy = float(tp + tn) / (tp + fp + tn + fn)
    return recall, precision, accuracy


# Spell checking tools

In [111]:
import spellchecker
from spello.model import SpellCorrectionModel
import hunspell
import jamspell
import textblob
import autocorrect

Spell checker classes creation:

In [112]:
class SpellCorrector(object):
    def __init__(self, name):
        self.__name = name

    def correct(self, sentence: list, position: int):
        pass

    def get_name(self):
        return self.__name


class PyspellcheckerCorrector(SpellCorrector):
    def __init__(self):
        super(PyspellcheckerCorrector, self).__init__("Pyspellchecker")
        self.__spellchecker = spellchecker.SpellChecker()

    def correct(self, sentence: list, position: int):
        word = sentence[position]
        self.__spellchecker.unknown([word])
        correction_candidates = self.__spellchecker.candidates(word)
        if correction_candidates is None:
            return word
        return list(correction_candidates)


class TextblobCorrector(SpellCorrector):
    def __init__(self):
        super(TextblobCorrector, self).__init__("Textblob")

    def correct(self, sentence: list, position: int):
        word_str = sentence[position]
        blob_word = textblob.Word(word_str)
        correction_candidates = blob_word.spellcheck()
        if len(correction_candidates) == 0:
            return word_str
        return [word_prob[0] for word_prob in correction_candidates]


class SpelloCorrector(SpellCorrector):
    def __init__(self, spello_model_path: str = "models/spello-en.pkl"):
        super(SpelloCorrector, self).__init__("Spello")
        self.__spellchecker = SpellCorrectionModel(language='en')
        self.__spellchecker.load(spello_model_path)

    def correct(self, sentence: list, position: int):
        word = sentence[position]
        correction_candidates = self.__spellchecker.suggest(word)
        if len(correction_candidates) == 0:
            return word
        return [word_prob[0] for word_prob in correction_candidates]


class HunspellCorrector(SpellCorrector):
    def __init__(self, hunspell_model_path: str = "models/hunspell-en_US/en_US"):
        super(HunspellCorrector, self).__init__("Hunspell")
        self.__spellchecker = hunspell.HunSpell(hunspell_model_path + '.dic',
                                                hunspell_model_path + '.aff')

    def correct(self, sentence: list, position: int):
        word = sentence[position]
        # spellchecker thinks that this word is correct
        if self.__spellchecker.spell(word):
            return word
        return self.__spellchecker.suggest(word)


class JamspellCorrector(SpellCorrector):
    def __init__(self, jamspell_model_path: str = "models/jamspell-en.bin"):
        super(JamspellCorrector, self).__init__("Jamspell")
        self.__spellchecker = jamspell.TSpellCorrector()
        self.__spellchecker.LoadLangModel(jamspell_model_path)

    def correct(self, sentence: list, position: int):
        correction_candidates = list(self.__spellchecker.GetCandidates(sentence, position))
        # spellchecker thinks that this word is correct
        if len(correction_candidates) == 0:
            return sentence[position]
        return correction_candidates


class AutocorrectCorrector(SpellCorrector):
    def __init__(self):
        super(AutocorrectCorrector, self).__init__("Autocorrect")
        self.__spellchecker = autocorrect.Speller()

    def correct(self, sentence: list, position: int):
        word = sentence[position]
        correction_candidates = self.__spellchecker.get_candidates(word)
        if len(correction_candidates) == 0:
            return word
        return [word_prob[1] for word_prob in correction_candidates]


The checker will alternately receive words from the misspelled sentence, we will see how it copes with its task and count metrics in parallel:

In [113]:
import time


def evaluate_checker(spellchecker: SpellCorrector, correct_sentence: str, misspell_sentence: str,
                     invalids_in_misspell_sentence: int):
    # values for metrics calculation
    invalid_recognized_misspelled = 0
    valid_recognized_misspelled = 0
    invalid_recognized_correct = 0
    valid_recognized_correct = 0

    fixed_invalids = 0
    broken_valids = 0
    not_fixed_invalids = 0
    not_fixed_but_correction_in_top5 = 0

    checker_worktime = 0

    correct_sentence_words = correct_sentence.split()
    misspell_sentence_words = misspell_sentence.split()

    n_words = len(misspell_sentence_words)
    for pos in range(n_words):
        misspell_sentence_word = misspell_sentence_words[pos]
        correct_sentence_word = correct_sentence_words[pos]

        # mark the time of operation of the correct function
        correction_start_time = time.time()
        checker_correction_candidates = spellchecker.correct(misspell_sentence_words, pos)
        checker_worktime += time.time() - correction_start_time

        if isinstance(checker_correction_candidates, list):
            word_recognized_correct = False
            checker_word_correction = checker_correction_candidates[0]
            top5_correction_candidates = checker_correction_candidates[:5] if \
                len(checker_word_correction) >= 5 else checker_correction_candidates
        else:
            word_recognized_correct = True
            checker_word_correction = checker_correction_candidates
            top5_correction_candidates = [checker_word_correction]

        # the word was originally correct
        if misspell_sentence_word == correct_sentence_word:
            # checker recognized word as correct
            if word_recognized_correct:
                valid_recognized_correct += 1
            else:
                # checker recognized word as misspelled
                valid_recognized_misspelled += 1
                # if checker suggested the wrong replacement, it brakes out the word
                if checker_word_correction != correct_sentence_word:
                    broken_valids += 1
        else:
            # the word was originally misspelled
            # checker recognized word as correct
            if word_recognized_correct:
                invalid_recognized_correct += 1
            else:
                # checker recognized word as misspelled
                invalid_recognized_misspelled += 1
                # if checker suggested the correct replacement, it fixes the word
                if checker_word_correction == correct_sentence_word:
                    fixed_invalids += 1
                else:
                    not_fixed_invalids += 1
                    # if the word wasn't fixed, but right correction was in top5 candidates
                    if correct_sentence_word in top5_correction_candidates:
                        not_fixed_but_correction_in_top5 += 1

    misspells_after_checker = invalids_in_misspell_sentence - fixed_invalids + broken_valids

    # calculating metrics
    recall, precision, accuracy = calculate_main_metrics(invalid_recognized_misspelled,
                                                         valid_recognized_misspelled,
                                                         invalid_recognized_correct,
                                                         valid_recognized_correct)
    misspells_after_checker_percent = float(misspells_after_checker) / n_words
    fixed_invalids_percent = float(fixed_invalids) / invalids_in_misspell_sentence
    not_fixed_invalids_but_correction_in_top5_percent = float(not_fixed_but_correction_in_top5) / not_fixed_invalids
    broken_valids_percent = float(broken_valids) / (n_words - invalids_in_misspell_sentence)
    speed = float(n_words) / checker_worktime

    return recall * 100, \
           precision * 100, \
           accuracy * 100, \
           misspells_after_checker_percent * 100, \
           fixed_invalids_percent * 100, \
           not_fixed_invalids_but_correction_in_top5_percent * 100, \
           broken_valids_percent * 100, \
        speed

In [114]:
def calculate_metrics_and_print(spellchecker: SpellCorrector, misspells_dict: dict):
    correct_sentence, misspell_sentence, invalids_in_misspell_sentence = build_correct_and_misspell_sentence(
        misspells_dict)

    recall, \
        precision, \
        accuracy, \
        misspells_after_checker_percent, \
        fixed_invalids_percent, \
        not_fixed_invalids_but_correction_in_top5_percent, \
        broken_valids_percent, \
        speed = evaluate_checker(spellchecker, correct_sentence, misspell_sentence, invalids_in_misspell_sentence)

    print("Checker: {0}\n"
          "Classifying recall: {1:.2f} %\n"
          "Classifying precision: {2:.2f} %\n"
          "Classifying accuracy: {3:.2f} %\n"
          "Misspells after checker percent: {4:.2f} %\n"
          "Fixed misspellings percent: {5:.2f} %\n"
          "Not fixed but may be corrected by one in top-5 percent: {6:.2f} %\n"
          "Broken valids percent: {7:.2f} %\n"
          "Speed: {8:.7f} words/ sec\n"
          "----------------------------".format(spellchecker.get_name(),
                                                recall,
                                                precision,
                                                accuracy,
                                                misspells_after_checker_percent,
                                                fixed_invalids_percent,
                                                not_fixed_invalids_but_correction_in_top5_percent,
                                                broken_valids_percent,
                                                speed))

In [115]:
def compare_spellcheckers():
    path_to_misspells_dataset = "data/wikipedia_misspells.txt"
    misspells_dict = read_misspells_dataset(path_to_misspells_dataset)

    correctors = [PyspellcheckerCorrector(),
                  SpelloCorrector(),
                  HunspellCorrector(),
                  JamspellCorrector(),
                  TextblobCorrector(),
                  AutocorrectCorrector()
                  ]

    for corrector in correctors:
        calculate_metrics_and_print(corrector, misspells_dict)

In [116]:
compare_spellcheckers()

Checker: Pyspellchecker
Classifying recall: 98.64 %
Classifying precision: 55.87 %
Classifying accuracy: 56.04 %
Misspells after checker percent: 21.34 %
Fixed misspellings percent: 64.48 %
Not fixed but may be corrected by one in top-5 percent: 57.58 %
Broken valids percent: 3.23 %
Speed: 33.4629661 words/ sec
----------------------------
Symspell suggestions: [('pennies', 2), ('pennies', 2)]
Phoneme suggestions: []
Symspell suggestions: [('pennies', 2), ('openings', 2), ('penises', 2), ('pennies', 2), ('openings', 2), ('penises', 2)]
Phoneme suggestions: []
Symspell suggestions: [('pipelines', 3), ('happenings', 3), ('pennies', 3), ('openings', 3), ('appendices', 3), ('penises', 3), ('appetites', 3), ('pipelines', 3), ('happenings', 3), ('pennies', 3), ('openings', 3), ('appendices', 3), ('penises', 3), ('appetites', 3)]
Phoneme suggestions: []
Symspell suggestions: [('athenian', 1), ('athenian', 1)]
Phoneme suggestions: [('athenian', 1), ('athenian', 1)]
Symspell suggestions: [('ath

In [117]:
def func():
    a = 2
    print(a)


func()

2


## 1. Pyspellchecker library

### Work principles description

Spell checking [library](https://github.com/barrust/pyspellchecker) that implements [Peter Norvig's](https://norvig.com/spell-correct.html) algorithm idea, which many subsequently turned to for comparison or improvement.

**How does it work?**
Clipping from the library description:

"It uses a [Levenshtein Distance](https://en.wikipedia.org/wiki/Levenshtein_distance) algorithm to find permutations within an edit distance of 2 from the original word. It then compares all permutations (insertions, deletions, replacements, and transpositions) to known words in a word frequency list. Those words that are found more often in the frequency list are more likely the correct results."

## 2. Pre-trained spell correction model 'Spello'

## 3. Hunspell library

## 4. JamSpell library

## 5. Textblob library

## 6. Autocorrect library